In [1]:
# Import libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
import lifelines

# Import and tidy data
%run tidy_data.py

# Risk evaluation
%run risks.py

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 174 non-null    object  
 1   age                174 non-null    int64   
 2   variant_histology  174 non-null    category
 3   ypT_stage          168 non-null    category
 4   ypN_stage          174 non-null    category
 5   recurrence         173 non-null    category
 6   fu_recurrence      174 non-null    int64   
 7   dod                171 non-null    category
 8   doc                170 non-null    category
 9   fu_censor          174 non-null    int64   
 10  sex                174 non-null    object  
 11  year_cystectomy    173 non-null    float64 
 12  ypT_group1         168 non-null    category
 13  ypT_group2         168 non-null    category
 14  ypN_group1         174 non-null    category
 15  ypN_group2         174 non-null    category
 16  dre     

# OR - Tumor recurrence

In [2]:
# Copy full dataframe and drop NAN values
df_recurrence = df.dropna(subset=['recurrence']).copy()

In [3]:
# Create dummy variables for predictors and outcome
df_recurrence = pd.get_dummies(df_recurrence, columns=['sex','variant_histology','ypT_group1','ypN_group1','recurrence'], drop_first=True)

# sex: Female vs Male
# variant_histology: No vs Yes
# ypT: pT0-pTis-pTa vs pT2-pT3
# ypN: pN0 vs pN1-pN2-pN3
df_recurrence = df_recurrence.rename(columns={'ypT_group1_pT2-pT3':'ypT','ypN_group1_pN1-pN2-pN3':'ypN','sex_Male':'sex','variant_histology_Yes':'variant_histology','recurrence_Yes':'recurrence'})
df_recurrence = df_recurrence[['age','sex','variant_histology','ypT','ypN','recurrence','fu_recurrence']]
df_recurrence.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 0 to 173
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   age                173 non-null    int64
 1   sex                173 non-null    uint8
 2   variant_histology  173 non-null    uint8
 3   ypT                173 non-null    uint8
 4   ypN                173 non-null    uint8
 5   recurrence         173 non-null    uint8
 6   fu_recurrence      173 non-null    int64
dtypes: int64(2), uint8(5)
memory usage: 4.9 KB


In [4]:
# age
logistic_regression(df_recurrence, 'age', 'recurrence')

Optimization terminated successfully.
         Current function value: 129.546509
         Iterations 5


,Odds Ratio,Lower CI,Upper CI,P-value
age,0.971051,0.939928,1.003205,0.077157


In [5]:
# sex
logistic_regression(df_recurrence, 'sex', 'recurrence')

Optimization terminated successfully.
         Current function value: 123.688049
         Iterations 5


,Odds Ratio,Lower CI,Upper CI,P-value
sex,1.382143,0.672737,2.839622,0.378347


In [6]:
# variant_histology
logistic_regression(df_recurrence, 'variant_histology', 'recurrence')

Optimization terminated successfully.
         Current function value: 121.870484
         Iterations 4


,Odds Ratio,Lower CI,Upper CI,P-value
variant_histology,1.027778,0.532254,1.984631,0.934958


In [7]:
# ypT
logistic_regression(df_recurrence, 'ypT', 'recurrence')

Optimization terminated successfully.
         Current function value: 127.799097
         Iterations 5


,Odds Ratio,Lower CI,Upper CI,P-value
ypT,0.535842,0.26049,1.102258,0.09


In [8]:
# ypN
logistic_regression(df_recurrence, 'ypN', 'recurrence')

Optimization terminated successfully.
         Current function value: 131.314498
         Iterations 5


,Odds Ratio,Lower CI,Upper CI,P-value
ypN,2.313889,1.084256,4.938023,0.030073


In [9]:
# Adjusted OR
logistic_regression(df_recurrence, ['age','sex','variant_histology','ypT','ypN'], 'recurrence')

Optimization terminated successfully.
         Current function value: 140.889200
         Iterations 6


,Odds Ratio,Lower CI,Upper CI,P-value
age,0.975404,0.943505,1.008381,0.142103
sex,1.313757,0.621729,2.776058,0.474663
variant_histology,1.148678,0.575754,2.291709,0.694069
ypT,1.651144,0.181170,15.048147,0.656482
ypN,3.707364,0.379528,36.214879,0.259820


# HR - Tumor recurrence

In [10]:
# age
cox_regression(df_recurrence, 'fu_recurrence', 'age', 'recurrence')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
age,0.967761,0.941088,0.995189,0.021551


In [11]:
# sex
cox_regression(df_recurrence, 'fu_recurrence', 'sex', 'recurrence')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
sex,1.226806,0.669816,2.246963,0.507946


In [12]:
# variant_histology
cox_regression(df_recurrence, 'fu_recurrence', 'variant_histology', 'recurrence')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
variant_histology,1.074032,0.625188,1.845116,0.795878


In [13]:
# ypT
cox_regression(df_recurrence, 'fu_recurrence', 'ypT', 'recurrence')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
ypT,0.813886,0.460209,1.43937,0.478983


In [14]:
# ypN
cox_regression(df_recurrence, 'fu_recurrence', 'ypN', 'recurrence')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
ypN,1.375456,0.769726,2.457861,0.281787


In [15]:
# Adjusted HR
cox_regression(df_recurrence, 'fu_recurrence', 'age + sex + variant_histology + ypT + ypN', 'recurrence')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
age,0.969185,0.941390,0.997801,0.035007
sex,1.243566,0.664281,2.328018,0.495640
variant_histology,1.163884,0.657068,2.061622,0.602880
ypT,1.506288,0.198786,11.413788,0.691767
ypN,1.879589,0.240570,14.685378,0.547417


# OR - Death by cancer

In [16]:
df_dod = df.dropna(subset=['dod']).copy()

In [17]:
# Create dummy variables for predictors and outcome
df_dod = pd.get_dummies(df_dod, columns=['sex','variant_histology','ypT_group1','ypN_group1','dod'], drop_first=True)

# sex: Female vs Male
# variant_histology: No vs Yes
# ypT: pT0-pTis-pTa vs pT2-pT3
# ypN: pN0 vs pN1-pN2-pN3
df_dod = df_dod.rename(columns={'ypT_group1_pT2-pT3':'ypT','ypN_group1_pN1-pN2-pN3':'ypN','sex_Male':'sex','variant_histology_Yes':'variant_histology','dod_Yes':'dod'})
df_dod = df_dod[['age','sex','variant_histology','ypT','ypN','dod','fu_censor']]
df_dod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 0 to 173
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   age                171 non-null    int64
 1   sex                171 non-null    uint8
 2   variant_histology  171 non-null    uint8
 3   ypT                171 non-null    uint8
 4   ypN                171 non-null    uint8
 5   dod                171 non-null    uint8
 6   fu_censor          171 non-null    int64
dtypes: int64(2), uint8(5)
memory usage: 4.8 KB


In [18]:
# age
logistic_regression(df_dod, 'age', 'dod')

Optimization terminated successfully.
         Current function value: 224.149439
         Iterations 5


,Odds Ratio,Lower CI,Upper CI,P-value
age,0.996748,0.962088,1.032656,0.856829


In [19]:
# sex
logistic_regression(df_dod, 'sex', 'dod')

Optimization terminated successfully.
         Current function value: 228.131299
         Iterations 5


,Odds Ratio,Lower CI,Upper CI,P-value
sex,0.638596,0.303365,1.344275,0.237631


In [20]:
# variant_histology
logistic_regression(df_dod, 'variant_histology', 'dod')

Optimization terminated successfully.
         Current function value: 224.387766
         Iterations 5


,Odds Ratio,Lower CI,Upper CI,P-value
variant_histology,1.130213,0.548852,2.327371,0.739788


In [21]:
# ypT
logistic_regression(df_dod, 'ypT', 'dod')

Optimization terminated successfully.
         Current function value: 226.440022
         Iterations 5


,Odds Ratio,Lower CI,Upper CI,P-value
ypT,0.693894,0.315123,1.527941,0.36421


In [22]:
# ypN
logistic_regression(df_dod, 'ypN', 'dod')

Optimization terminated successfully.
         Current function value: 225.523396
         Iterations 5


,Odds Ratio,Lower CI,Upper CI,P-value
ypN,1.354839,0.587449,3.124675,0.476298


In [23]:
# Adjusted OR
logistic_regression(df_dod, ['age','sex','variant_histology','ypT','ypN'], 'dod')

Optimization terminated successfully.
         Current function value: 230.877664
         Iterations 5


,Odds Ratio,Lower CI,Upper CI,P-value
age,0.996066,0.961011,1.032399,0.829254
sex,0.653852,0.305689,1.398555,0.273405
variant_histology,1.169000,0.555257,2.461136,0.681006
ypT,0.654361,0.109887,3.896622,0.641306
ypN,0.933437,0.140697,6.192784,0.943122


# HR - Death by cancer

In [24]:
# age
cox_regression(df_dod, 'fu_censor', 'age', 'dod')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
age,0.99342,0.958538,1.029571,0.717339


In [25]:
# sex
cox_regression(df_dod, 'fu_censor', 'sex', 'dod')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
sex,0.645013,0.339757,1.224529,0.180033


In [26]:
# variant_histology
cox_regression(df_dod, 'fu_censor', 'variant_histology', 'dod')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
variant_histology,1.264452,0.670888,2.383169,0.46808


In [27]:
# ypT
cox_regression(df_dod, 'fu_censor', 'ypT', 'dod')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
ypT,0.984809,0.491501,1.973238,0.965566


In [28]:
# ypN
cox_regression(df_dod, 'fu_censor', 'ypN', 'dod')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
ypN,0.93083,0.442735,1.957027,0.850051


In [29]:
# Adjusted HR
cox_regression(df_dod, 'fu_censor', 'age + sex + variant_histology + ypT + ypN', 'dod')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
age,0.991114,0.956427,1.027059,0.623390
sex,0.673720,0.347078,1.307771,0.243186
variant_histology,1.229283,0.632043,2.390877,0.543049
ypT,0.719642,0.160704,3.222598,0.667109
ypN,0.706043,0.144831,3.441928,0.666711


# OR - Death by other causes

In [30]:
# Copy dataset and drop NANs values
df_doc = df.dropna(subset=['recurrence']).copy()

In [31]:
# Create dummy variables for predictors and outcome
df_doc = pd.get_dummies(df_doc, columns=['sex','variant_histology','ypT_group1','ypN_group1','doc'], drop_first=True)

# sex: Female vs Male
# variant_histology: No vs Yes
# ypT: pT0-pTis-pTa vs pT2-pT3
# ypN: pN0 vs pN1-pN2-pN3
df_doc = df_doc.rename(columns={'ypT_group1_pT2-pT3':'ypT','ypN_group1_pN1-pN2-pN3':'ypN','sex_Male':'sex','variant_histology_Yes':'variant_histology','doc_Yes':'doc'})
df_doc = df_doc[['age','sex','variant_histology','ypT','ypN','doc','fu_censor']]
df_doc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 0 to 173
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   age                173 non-null    int64
 1   sex                173 non-null    uint8
 2   variant_histology  173 non-null    uint8
 3   ypT                173 non-null    uint8
 4   ypN                173 non-null    uint8
 5   doc                173 non-null    uint8
 6   fu_censor          173 non-null    int64
dtypes: int64(2), uint8(5)
memory usage: 4.9 KB


In [32]:
# age
logistic_regression(df_doc, 'age', 'doc')

Optimization terminated successfully.
         Current function value: inf
         Iterations 6


/Users/alcideschaux/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/alcideschaux/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


,Odds Ratio,Lower CI,Upper CI,P-value
age,1.025689,0.972561,1.08172,0.349947


In [33]:
# sex
logistic_regression(df_doc, 'sex', 'doc')

Optimization terminated successfully.
         Current function value: inf
         Iterations 7


/Users/alcideschaux/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/alcideschaux/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


,Odds Ratio,Lower CI,Upper CI,P-value
sex,3.588785,0.793678,16.22746,0.096953


In [34]:
# variant_histology
logistic_regression(df_doc, 'variant_histology', 'doc')

Optimization terminated successfully.
         Current function value: 497.122829
         Iterations 6


,Odds Ratio,Lower CI,Upper CI,P-value
variant_histology,0.643236,0.218115,1.896951,0.42391


In [35]:
# ypT
logistic_regression(df_doc, 'ypT', 'doc')

Optimization terminated successfully.
         Current function value: 492.316243
         Iterations 6


,Odds Ratio,Lower CI,Upper CI,P-value
ypT,1.097458,0.34029,3.539374,0.876302


In [36]:
# ypN
logistic_regression(df_doc, 'ypN', 'doc')

Optimization terminated successfully.
         Current function value: 493.367368
         Iterations 6


,Odds Ratio,Lower CI,Upper CI,P-value
ypN,0.76875,0.209681,2.818451,0.691551


In [37]:
# Adjusted OR
logistic_regression(df_doc, ['age','sex','variant_histology','ypT','ypN'], 'doc')

Optimization terminated successfully.
         Current function value: inf
         Iterations 7


/Users/alcideschaux/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/alcideschaux/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


,Odds Ratio,Lower CI,Upper CI,P-value
age,1.031829,0.972292,1.095011,0.301461
sex,4.316721,0.898089,20.748587,0.067884
variant_histology,0.640317,0.209429,1.957732,0.434326
ypT,0.394192,0.036203,4.292119,0.444777
ypN,0.307660,0.021871,4.327789,0.382194


# HR - Death by cancer

In [38]:
# age
cox_regression(df_doc, 'fu_censor', 'age', 'doc')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
age,1.030337,0.973804,1.090152,0.299269


In [39]:
# sex
cox_regression(df_doc, 'fu_censor', 'sex', 'doc')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
sex,3.145075,0.722594,13.688871,0.126766


In [40]:
# variant_histology
cox_regression(df_doc, 'fu_censor', 'variant_histology', 'doc')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
variant_histology,0.806683,0.284362,2.288411,0.68635


In [41]:
# ypT
cox_regression(df_doc, 'fu_censor', 'ypT', 'doc')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
ypT,1.526211,0.492345,4.731074,0.463904


In [42]:
# ypN
cox_regression(df_doc, 'fu_censor', 'ypN', 'doc')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
ypN,0.542646,0.15349,1.918467,0.342741


In [43]:
# Adjusted HR
cox_regression(df_doc, 'fu_censor', 'age + sex + variant_histology + ypT + ypN', 'doc')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
age,1.031526,0.972010,1.094686,0.305980
sex,3.635479,0.808680,16.343549,0.092362
variant_histology,0.801235,0.273694,2.345598,0.685954
ypT,0.547362,0.066599,4.498614,0.574971
ypN,0.283499,0.026472,3.036120,0.297429


# OR - Disease-related event

In [44]:
df_dre = df.dropna(subset=['dre']).copy()

In [45]:
# Create dummy variables for predictors and outcome
df_dre = pd.get_dummies(df_dre, columns=['sex','variant_histology','ypT_group1','ypN_group1','dre'], drop_first=True)

# sex: Female vs Male
# variant_histology: No vs Yes
# ypT: pT0-pTis-pTa vs pT2-pT3
# ypN: pN0 vs pN1-pN2-pN3
df_dre = df_dre.rename(columns={'ypT_group1_pT2-pT3':'ypT','ypN_group1_pN1-pN2-pN3':'ypN','sex_Male':'sex','variant_histology_Yes':'variant_histology','dre_Yes':'dre'})
df_dre = df_dre[['age','sex','variant_histology','ypT','ypN','dre','fu_censor']]
df_doc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 0 to 173
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   age                173 non-null    int64
 1   sex                173 non-null    uint8
 2   variant_histology  173 non-null    uint8
 3   ypT                173 non-null    uint8
 4   ypN                173 non-null    uint8
 5   doc                173 non-null    uint8
 6   fu_censor          173 non-null    int64
dtypes: int64(2), uint8(5)
memory usage: 9.0 KB


In [46]:
# age
logistic_regression(df_dre, 'age', 'dre')

Optimization terminated successfully.
         Current function value: 53.412956
         Iterations 4


,Odds Ratio,Lower CI,Upper CI,P-value
age,0.989306,0.959376,1.02017,0.492759


In [47]:
# sex
logistic_regression(df_dre, 'sex', 'dre')

Optimization terminated successfully.
         Current function value: 57.267896
         Iterations 4


,Odds Ratio,Lower CI,Upper CI,P-value
sex,1.75,0.876447,3.494221,0.112701


In [48]:
# variant_histology
logistic_regression(df_dre, 'variant_histology', 'dre')

Optimization terminated successfully.
         Current function value: 53.220644
         Iterations 4


,Odds Ratio,Lower CI,Upper CI,P-value
variant_histology,0.807692,0.429263,1.519738,0.50783


In [49]:
# ypT
logistic_regression(df_dre, 'ypT', 'dre')

Optimization terminated successfully.
         Current function value: 62.008083
         Iterations 4


,Odds Ratio,Lower CI,Upper CI,P-value
ypT,0.520263,0.256542,1.055083,0.07009


In [50]:
# ypN
logistic_regression(df_dre, 'ypN', 'dre')

Optimization terminated successfully.
         Current function value: 65.977874
         Iterations 4


,Odds Ratio,Lower CI,Upper CI,P-value
ypN,2.230769,1.05115,4.73418,0.036627


In [51]:
# Adjusted OR
logistic_regression(df_dre, ['age','sex','variant_histology','ypT','ypN'], 'dre')

Optimization terminated successfully.
         Current function value: 70.521640
         Iterations 5


,Odds Ratio,Lower CI,Upper CI,P-value
age,0.993835,0.962637,1.026045,0.703946
sex,1.751868,0.858582,3.574548,0.123336
variant_histology,0.917842,0.476643,1.767428,0.797617
ypT,0.945118,0.160445,5.567306,0.950256
ypN,2.073147,0.320889,13.393845,0.443740


# HR - Disease-related event

In [52]:
# age
cox_regression(df_dre, 'fu_censor', 'age', 'dre')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
age,0.990577,0.963696,1.018207,0.499985


In [53]:
# sex
cox_regression(df_dre, 'fu_censor', 'sex', 'dre')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
sex,1.28133,0.733732,2.237612,0.383478


In [54]:
# variant_histology
cox_regression(df_dre, 'fu_censor', 'variant_histology', 'dre')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
variant_histology,1.061362,0.646682,1.741951,0.813755


In [55]:
# ypT
cox_regression(df_dre, 'fu_censor', 'ypT', 'dre')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
ypT,0.975075,0.581592,1.634775,0.923731


In [56]:
# ypN
cox_regression(df_dre, 'fu_censor', 'ypN', 'dre')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
ypN,1.068997,0.626855,1.822995,0.806458


In [57]:
# Adjusted HR
cox_regression(df_dre, 'fu_censor', 'age + sex + variant_histology + ypT + ypN', 'dre')

,Hazards Ratio,Lower CI,Upper CI,P-value
covariate,,,,
age,0.991471,0.963951,1.019777,0.550899
sex,1.283087,0.721314,2.282380,0.396290
variant_histology,1.120378,0.663322,1.892365,0.670819
ypT,1.048496,0.243728,4.510535,0.949277
ypN,1.105296,0.246831,4.949462,0.895866
